<a href="https://colab.research.google.com/github/epaul-eki/WQTC25_workshop_IMTR/blob/main/water_quality_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Water Quality Data Visualization using Google Colab and R**
## AWWA WQTC 2025 Data Analysis and Coding Workshop

---

This notebook walks through building and running an interactive **Shiny** dashboard inside **Google Colab** using the **R** programming language. Users will learn how to load, inspect, and analyze data from a spreadsheet, generate visualizations of time series plots and maps, and develop an easy-to-use dashboard that synthesizes data into single webpage.

The data used in this notebook is publicly available at Thurston County:
1.   Water Quality Data: https://github.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/blob/main/wqp_data.csv
2.   Stream Site Locations and Aquatic Use Data: https://github.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/blob/main/outputs/streams_sites.csv

Thurston County Streams Water Quality Dashboard: https://www.thurstoncountywa.gov/streams-water-quality-dashboard

---

Notebook Table of Contents:
1.   Setup R and Install Packages
2.   Load R Libraries
3.   Load and Inspect Data
4.   Data Cleaning and Processing
5.   Generating Plots
6.   Generating Statistics
7.   Generating Maps

## 1. Setup R and Install Packages
Prior to installing any packages, please change the **runtime type** to **R**.



In [ ]:
# Install necessary packages (may take 6-8 minutes)
install.packages(c("shiny", "ggplot2", "dplyr", "lubridate", "leaflet"))

## 2. Load R Libraries

In [ ]:
library(shiny)
library(ggplot2)
library(dplyr)
library(lubridate)
library(leaflet)

## 3. Load and Inspect Data

In [ ]:
# URLs of water quality data (Thurston County)
sites_url <- "https://raw.githubusercontent.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/main/outputs/streams_sites.csv"
data_url <- "https://raw.githubusercontent.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/main/wqp_data.csv"

In [ ]:
# Read in stream site location data
sites <- read.csv(sites_url)
print("This is the stream site location data:")
head(sites)

In [ ]:
# Read in water quality data
data <- read.csv(data_url)
print("This is the water quality data associated with each stream site:")
data

## 4. Data Cleaning and Processing

In [ ]:
# Convert date_time column to a "date" data type
data$date_time <- parse_date_time(data$date_time, orders = c("ymd", "ymd HMS")) # since the dates are in multiple formats, we need to define those

head(data)

In [ ]:
# Join the two dataframes based on a common column (e.g., SITE_NAME)
data <- data %>%
  left_join(sites %>% select(SITE_NAME, AquaticLifeUse), by = "SITE_NAME") # select the columns from the stream site data that you want to keep

head(data)

In [ ]:
# Stream site names
site_names <- unique(data$SITE_NAME)
site_names

In [ ]:
# Water quality parameters
param_names <- unique(data$parameter)
param_names

## 5. Generating Plots

In [ ]:
# Choose a site and parameter
example_site <- site_names[47]
print(paste('The site name is:', example_site))

example_param <- param_names[3]
print(paste('The parameter name is:', example_param))

In [ ]:
# Filter data based on selections above
filtered_data <- data %>%
  filter(SITE_NAME == example_site, parameter == example_param)
filtered_data

In [ ]:
# Generate a basic plot
ggplot(data = filtered_data, aes(x = date_time, y = value)) +
  geom_line(color = "blue")

In [ ]:
# Edit the plot
ggplot(data = filtered_data, aes(x = date_time, y = value)) +
  geom_line(color = "blue") +
  geom_point(color = "darkblue") + # add points
  labs(title = paste(example_param, "at", example_site), # plot title
        x = "Date", # x-axis title
        y = paste0(example_param, " (", unique(filtered_data$unit), ")")) + # y-axis title
  theme(
    plot.title = element_text(size = 16), # plot title font size
    axis.title = element_text(size = 14), # axis title font size
    axis.text = element_text(size = 14) # axis text (values and dates) font size
  )

In [ ]:
# Use a for loop to generate plots of stream temperature at each site location and save as a pdf


## 6. Generating Statistics

In [ ]:
# Generate some basic stats of the example site and parameter
mean <- mean(filtered_data$value)
min <- min(filtered_data$value)
max <- max(filtered_data$value)

paste('The MEAN', example_param, 'at', example_site, 'is:', mean, unique(filtered_data$unit))
paste('The MIN', example_param, 'at', example_site, 'is:', min, unique(filtered_data$unit))
paste('The MAX', example_param, 'at', example_site, 'is:', max, unique(filtered_data$unit))

In [ ]:
# Generate a summary table of stats
summary_table <- filtered_data %>%
  summarise(
    Records = n(), # number of records
    Mean = round(mean(value, na.rm = TRUE), 2), # mean rounded to 2 decimals
    Min = round(min(value, na.rm = TRUE), 2), # min rounded to 2 decimals
    Max = round(max(value, na.rm = TRUE), 2), # max rounded to 2 decimals
    First_Record = min(date_time),
    Last_Record = max(date_time)
  )

summary_table


## 7. Generating Maps